In [280]:
import pystan
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd

import datetime
from datetime import date

from scipy.stats import norm
from util import util

import os
os.environ['STAN_NUM_THREADS'] = "4"

In [281]:
# We import the sample_submission.csv file as a way of determining
# the order of the rows in out output file
sample_submission = pd.read_csv("../sample_submission.csv")

# The fips_key.csv file contains standard information about each county
key = pd.read_csv("../data/us/processing_data/fips_key.csv", encoding='latin-1')

# Daily deaths contains the death count per day for each county.
# Cumulative deaths contains the total death count for each county
# by day.
daily_deaths = pd.read_csv("../data/us/covid/nyt_us_counties_daily.csv")
cumulative_deaths = pd.read_csv("../data/us/covid/deaths.csv")
county_land_areas = pd.read_csv("../data/us/demographics/county_land_areas.csv", encoding='latin1')
county_population = pd.read_csv("../data/us/demographics/county_populations.csv", encoding='latin1')
mobility_data = pd.read_csv("../data/google_mobility/Global_Mobility_Report.csv", encoding='latin1')

# List of all counties
all_fips = key["FIPS"].tolist()

util = util(daily_deaths, cumulative_deaths, county_land_areas, county_population, mobility_data, key)

In [282]:
model = """
data {
    int<lower=1> M;                    // Number of counties
    int<lower=1> N;                 // Days of observed data
    int<lower=1> predict_days;         // Number of days to predict
    
    int reported_cases[M, N];
    int reported_deaths[M, N];
    
    real pi[N];
}

parameters {
    real<lower=0> death_rate[M];
    real<lower=0> phi[M];
}

transformed parameters {
    matrix[M, N + predict_days] E_deaths;
    
    for (m in 1:M) {
        for (i in 1:19) {
            E_deaths[m, i] = 1e-9;
        }
    }
    
    for (m in 1:M) {
        for (i in 20:N) {
            E_deaths[m, i] = 1e-9;
            for (j in 1:(i-1)) {
                E_deaths[m, i] += reported_cases[m, j] * pi[i - j] * death_rate[m];
            }
        }
    }
    
    for (m in 1:M) {
        for (i in (N+1):(N+predict_days)) {
            E_deaths[m, i] = 1e-9;
            for (j in 20:N) {
                E_deaths[m, i] += reported_cases[m, j] * pi[i - j] * death_rate[m];
            }
        }
    }
}

model {
    for (m in 1:M) {
        death_rate[m] ~ normal(0.1, 0.05);
        phi[m] ~ normal(0,5);
    }
    
    for (m in 1:M) {
        for (i in 20:N) {
            reported_deaths[m, i] ~ neg_binomial_2(E_deaths[m, i], phi[m]);
        }
    }   
}
"""

In [283]:
extra_compile_args = ['-pthread', '-DSTAN_THREADS']
cases_model = pystan.StanModel(model_code=model, extra_compile_args=extra_compile_args)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09a2e01a82b24cf2e68c80bc9db06884 NOW.


In [286]:
df = pd.read_csv("AdjustedPi.csv")
pi_values = list(df.values.reshape(1, 200)[0])

def construct_data_for_pystan(end_date, fips_list, num_pred_days, num_past_days):
    data = {}
    data["M"] = len(fips_list)
    data["N"] = num_past_days
    data["predict_days"] = num_pred_days
    data["pi"] = pi_values[:data["N"]]
    
    reported_cases = []
    reported_deaths = []
    
    for fips in fips_list:
        cases = util.get_cases_list(fips, end_date)[-num_past_days:]
        deaths = util.get_deaths_list(fips, end_date)[-num_past_days:]
        
        reported_cases.append([int(x) for x in cases])
        reported_deaths.append([int(x) for x in deaths])
    
    data["reported_cases"] = reported_cases
    data["reported_deaths"] = reported_deaths
    
    return data

In [292]:
data = construct_data_for_pystan("2020-05-20", [53033], 14, 50)
control = {'max_treedepth': 20, 'adapt_delta': 0.8}
fit = cases_model.sampling(data=data, iter=400, chains=4, warmup=200, thin=4, control=control, n_jobs=8)

In [293]:
print(fit)

Inference for Stan model: anon_model_09a2e01a82b24cf2e68c80bc9db06884.
4 chains, each with iter=400; warmup=200; thin=4; 
post-warmup draws per chain=50, total post-warmup draws=200.

                 mean se_mean      sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
death_rate[1]    0.05  4.2e-4  5.7e-3   0.05   0.05   0.05   0.06   0.07    185   1.03
phi[1]           6.57    0.15    2.45   2.45   4.62   6.18   8.24  12.11    251   0.99
E_deaths[1,1]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,2]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,3]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,4]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,5]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,6]  1.0e-9 1.4e-25 2.1e-25  1e-09  1e-09  1e-09  1e-09  1e-09      2   0.98
E_deaths[1,7]  1.0e-9 1.4e-25 2.1